In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import requests
import time
import math
import hashlib
from lxml import html
# import pickle
from datetime import datetime
import pandas as pd
# import numpy as np
from bs4 import BeautifulSoup 

sys.setrecursionlimit(25000)
# sys.path.append('../python')
# import monitor_url as murl

In [3]:
df = pd.read_csv('../datasets/compiled_company_info.csv')
df = df.drop_duplicates()
df.head(len(df))

,Company,Yahoo Listed Co.,Symbol,Exchange,Market Cap,Company Size,Is American,Home URL,Press Release URL
0,AstraZeneca,AstraZeneca PLC,AZN,NYSE,"$140,554,649,600",v-large,Y,http://www.astrazeneca.com,https://www.astrazeneca.com/media-centre/press...
1,CanSino Biological Inc.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Beijing Institute of Biotechnology,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Moderna,"Moderna, Inc.",MRNA,NASDAQ,"$21,742,589,952",medium,Y,http://www.modernatx.com,https://investors.modernatx.com/news-releases/
4,NIAID,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,Wuhan Institute of Biological Products,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,Sinopharm,"Sinopharm Group Co., Ltd.",SHTDY,OTC Markets,"$8,092,427,776",small,N,http://www.sinopharmgroup.com.cn,http://ir.sinopharmgroup.com.cn/
7,Beijing Institute of Biological Products,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,Sinovac,Sinovac Biotech Ltd.,SVA,NASDAQ,"$460,249,280",small,Y,http://www.sinovacbio.com,http://www.sinovacbio.com/?optionid=754
9,Novavax,"Novavax, Inc.",NVAX,NASDAQ,"$4,791,271,936",small,Y,http://www.novavax.com,https://ir.novavax.com/press-releases


# Monitoring for changes
~~ As a starting point, looking into leveraging the URLWATCH package:
https://github.com/thp/urlwatch ~~

~~ To do from scratch, here are some references: ~~
- ~~https://chrisalbon.com/python/web_scraping/monitor_a_website/ ~~
- ~~https://www.adventuresintechland.com/detect-when-a-webpage-changes-with-python/~~
- ~~https://www.codementor.io/@gergelykovcs/how-and-why-i-built-a-simple-web-scrapig-script-to-notify-us-about-our-favourite-food-fcrhuhn45 ~~

Went the BeautifulSoup route

In [ ]:
page = requests.get('https://investors.biontech.de/press-releases/').text
soup = BeautifulSoup(page, 'html.parser')
body = soup.find('body')
print(body)

In [ ]:
# Testing with local html files
pg_original = '../datasets/original_webpage.html'
pg_updated = '../datasets/updated_webpage.html'

soup_original = BeautifulSoup(open(pg_original), 'html.parser')
soup_updated = BeautifulSoup(open(pg_updated), 'html.parser')


body_original = soup_original.find('body')
body_updated = soup_updated.find('body')

links_original = list(body_original.find_all('a'))
links_updated = list(body_updated.find_all('a'))

# print(f'Originally: {len(links_original)} links \n\
# Href: {len(links_href)} links \n')

# Method from
# https://stackoverflow.com/questions/41125909/python-find-elements-in-one-list-that-are-not-in-the-other
links_diff = list(set(links_updated) - set(links_original))

for link in links_diff:
    print(link)
    ref = link.get('href')
    content = link.text
    
    print(f'{content}: {ref} \n')
    #print(f'{link.get('href')}\n')

In [90]:
def url_to_filename(base_url:str):
    base_url = prune_url(base_url)
    
    forbidden_ascii = ['/', '\\', '\|', ':', '?',
                      '\'', '\"', '?', '*', '>', '<']
    
    for ascii_char in forbidden_ascii:
        base_url = base_url.replace(ascii_char, '-')
    
    base_url = base_url.replace('.', '_')
    
    return base_url
    
    
def prune_url(full_url:str):
    cut_chars = ['https://', 'http://', 'www.']
    
    for cut in cut_chars:
        full_url = full_url.replace(cut, '')
    
    if full_url[-1] == '/':
        pruned_url = full_url[0:-1]
    else:
        pruned_url = full_url
    
    return pruned_url



def now_hms():
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    return current_time


def load_cache(base_url, path='logs/'):
    url_filename = url_to_filename(base_url)
    filename = f'{path}CACHE-{url_filename}.html'
    f=open(filename, 'r')
    data_cache = f.read()
    data_cache = " ".join(data_cache.split())
    return data_cache


def store_cache(base_url, data, path='logs/'):
    url_filename = url_to_filename(base_url)
    filename = f'{path}CACHE-{url_filename}.html'
    f = open(filename, 'w')
    f.write(data)
    f.close()
                      

def cache_updated(base_url, data, path='logs/'):
    # Check cache exists
    url_filename = url_to_filename(base_url)
    filename = f'{path}CACHE-{url_filename}.html'
    cache_exists = os.path.isfile(filename)
    
    if cache_exists:
        data_cache = load_cache(base_url, path=path)
        f=open(filename, 'r')
        data_cache = f.read()
        
        soup_cache = BeautifulSoup(data_cache, 'html.parser')
        body_cache = soup_cache.find('body')
        soup_data = BeautifulSoup(data, 'html.parser')
        body_data = soup_data.find('body')
        
        hash_cache = hash(str(body_cache))
        hash_data = hash(str(body_data))
        
        
        if body_cache == body_data:
            message = f'No update from {base_url} \n'
            write_log(message, base_url, path=path)
            return False
        else:
            message = f'Update detected from {base_url} \n'
            write_log(message, base_url, path=path)
            return True
        
    else:
        message1 = f'No cached webpage found for {base_url} \n'
        message2 = f'Initializing page in {filename} \n'
        write_log(f'{message1}{message2}', base_url, path=path)
        store_cache(base_url, data, path=path)
        return False
        
        
        
        

def load_scache(base_url, path='logs/'):
    url_filename = url_to_filename(base_url)
    filename = f'{path}CACHE-{url_filename}.pkl'
    f=open(filename, 'rb')
    serial_data = f.read()
    data_cache1 = pickle.loads(serial_data)
    data_cache = pickle.loads(data_cache1) # Need to revisit
    return data_cache


def store_scache(base_url, data, path='logs/'):
    url_filename = url_to_filename(base_url)
    filename = f'{path}CACHE-{url_filename}.pkl'
    f = open(filename, 'wb')
    serial_data = pickle.dumps(data)
    pickle.dump(serial_data, f)
    # f.write(data_ser)
    f.close()
                      

def scache_updated(base_url, data, path='logs/'):
    # Check cache exists
    url_filename = url_to_filename(base_url)
    filename = f'{path}CACHE-{url_filename}.pkl'
    cache_exists = os.path.isfile(filename)
    
    if cache_exists:
        data_cache = load_cache(base_url, path=path)
        f=open(filename, 'rb')
        serial_cache = f.read()
        serial_data = pickle.dumps(data)
        
        if serial_data == serial_cache:
            message = f'No update from {base_url} \n'
            write_log(message, base_url, path=path)
            return False
        else:
            message = f'Update detected from {base_url} \n'
            write_log(message, base_url, path=path)
            return True
        
    else:
        message1 = f'No cached webpage found for {base_url} \n'
        message2 = f'Initializing page in {filename} \n'
        write_log(f'{message1}{message2}', base_url, path=path)
        store_cache(base_url, data, path=path)
        return False
        

def write_log(message:str, base_url, path='logs/'):
    print(message)
    url_filename = url_to_filename(base_url)
    filename = f'{path}LOG-{url_filename}.txt'
    f = open(filename, 'a')
    f.write(message)
    f.close()
    
def get_links(html_data):
    anchors = get_anchors(html_data)
    hrefs, content = parse_anchors(anchors)
    
    return hrefs, content, anchors
    

def get_anchors(html_data):
    if html_data.find('body') == None:
        # print('No body found, will pass original argument')
        html_body = html_data
    else:
        html_body = html_data.find('body')   
        
    anchors = list(html_body.find_all('a'))

    # anchors = simplify_anchors(anchors)
        
    return anchors

def simplify_anchors(anchors):
    for i, anchor in enumerate(anchors):
        anchor = str(anchor)
        while '  ' in anchor or '\n' in anchor or '\t' in anchor:
            anchor = anchor.replace('  ', ' ').replace('\n', ' ')
        
        #anchors[i] = BeautifulSoup(anchor, 'html.parser')
        anchors[i] = anchor
    return anchors

def parse_anchors(anchors):
    hrefs = []
    content = []
    
    for anchor in anchors:
        hrefs.append(anchor.get('href'))
        content.append(anchor.text)
    
    return hrefs, content

def get_new_diff(new_data, old_data):
    diff = list(set(new_data) - set(old_data))
    return diff


def is_na(subject):
    if isinstance(subject, str):
        na_versions = ['n/a', 'nan']
        if subject.lower() in na_versions:
            return True
        else: 
            return False
    elif isinstance(subject, float):
        if math.isnan(subject):
            return True
    elif isinstance(subject, bool):
        return subject
    else:
        return False
        
    
        

In [93]:
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}

for ii in range(len(df)): 
    co, yco, sym, exch, mkcap, size, am, url_home, url_pr = \
    df.iloc[ii].T.values
    
    if not is_na(url_pr):
        print('\n----------------------------------')
        print(f'Checking {yco}')
        
        try:
            r = requests.get(url_pr, headers=headers)
        except:
            message = f'CONNECTION FAILED! - Status Code: {r.status_code}\n'
            print(f'\n{message}')
            write_log(message, url_pr)
            continue
            
        page = r.text
        page = " ".join(page.split())
        soup = BeautifulSoup(page, 'html.parser')
        body = soup.find('body')
        
        if not body:
            message = f'Cannot find <body></body> for {url_pr}'
            write_log(message, url_pr)
            continue
            
        anchors = get_anchors(soup)
        
        has_update = cache_updated(url_pr, page)
        current_time = now_hms()
        
        print(has_update)
        print('\n')
     
        if has_update:
            old_page = load_cache(url_pr)
            old_soup = BeautifulSoup(old_page, 'html.parser')
            old_body = old_soup.find('body')
            old_anchors = get_anchors(old_soup)
            diff_anchors = get_new_diff(anchors, old_anchors)
            diff_hrefs, diff_contents = parse_anchors(diff_anchors)
            
            if len(diff_anchors) > 0:
                message = f'[{current_time}] New links found: \n---------------'
                write_log(message, url_pr)
                
                for diff_href, diff_content in zip(diff_hrefs, diff_contents):
                    message = f'{diff_href}  ::  {diff_content}'
                    write_log(message, url_pr)
                    
            else:
                message = f'[{current_time}] Update detected but no new anchors'
                write_log(message, url_pr)
                
            
            store_cache(url_pr, page)
            print('\n\n')
    
    else:
        print('\n----------------------------------')
        print(f'Skipping {co}')
        print('\n----------------------------------\n\n')

print('-----------------\nSweep Complete')
            


----------------------------------
Checking AstraZeneca PLC
No update from https://www.astrazeneca.com/media-centre/press-releases.html 

False



----------------------------------
Skipping CanSino Biological Inc.

----------------------------------



----------------------------------
Skipping Beijing Institute of Biotechnology

----------------------------------



----------------------------------
Checking Moderna, Inc.
Update detected from https://investors.modernatx.com/news-releases/ 

True


[14:45:52] Update detected but no new anchors




----------------------------------
Skipping NIAID

----------------------------------



----------------------------------
Skipping Wuhan Institute of Biological Products

----------------------------------



----------------------------------
Checking Sinopharm Group Co., Ltd.
Cannot find <body></body> for http://ir.sinopharmgroup.com.cn/

----------------------------------
Skipping Beijing Institute of Biological Products

-----------

In [ ]:
# print(url_pr)
#page = requests.get(url_pr, headers=headers)
#print(page.text)
# print(body)
# print(len(body.find_all('a')))
# store_cache(url_pr, page)

# soup_ = BeautifulSoup(page, 'html.parser')
# body_ = soup_.find_all('a')
# print(hash(str(body_)))


# headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_11_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/50.0.2661.102 Safari/537.36'}
# page = requests.get(url_pr, headers = headers)
# print(page)
# soup = BeautifulSoup(page.text, 'html.parser')
# body = soup.find_all('body')
# print(body)

# print(soup.prettify())
# cached = load_cache(url_pr)
# print(cached)

# url_filename = url_to_filename(url_pr)
# filename = f'logs/CACHE-{url_filename}.pkl'
# f=open(filename, 'rb')
# serial_data = f.read()
# data_cached = pickle.loads(serial_data)
# cached = pickle.loads(data_cached)
# print(data_cached)

In [8]:
url_pr = df.loc[df['Company'] == 'GSK']['Press Release URL'].T.values[0]
print(url_pr)
snap1 = requests.get(url_pr, headers = headers).text
soup1 = BeautifulSoup(snap1, 'html.parser')
print('Starting 10 second sleep')
time.sleep(10)
print('Sleep Finished')
snap2 = requests.get(url_pr, headers = headers).text
soup2 = BeautifulSoup(snap2, 'html.parser')

https://www.gsk.com/en-gb/media/press-releases/
Starting 10 second sleep
Sleep Finished


In [9]:
print('Getting anchors')
anchors1 = get_anchors(soup1)
print('Finished getting first batch of anchors')
anchors2 = get_anchors(soup2)
print('Finished getting second batch of anchors')

href1, content1 = parse_anchors(anchors1)
href2, content2 = parse_anchors(anchors2)
print('Data acquired')

Getting anchors
Finished getting first batch of anchors
Finished getting second batch of anchors
Data acquired


In [ ]:
# soup_body = soup1.find('body')   
# anchors = list(soup_body.find_all('a'))
# for i, _ in enumerate(anchors):
#     while '  ' in anchors[i] or '\t' in anchors[i] or '\n' in anchors[i]:
#         anchors[i] = anchors[i].replace('  ', ' ')
#         anchors[i] = anchors[i].replace('\t', ' ')
#         anchors[i] = anchors[i].replace('\n', ' ')
        
# print(anchors)

In [10]:
diff_anchors = get_new_diff(anchors2, anchors1)
print(diff_anchors)

test_load = load_cache(url_pr)
test_soup = BeautifulSoup(test_load, 'html.parser')
test_body = test_soup.find('body')
test_anchors = get_anchors(test_soup)
test_diff_anchors = get_new_diff(anchors2, test_anchors)
print(len(test_diff_anchors))
print(len(anchors2))
print(len(anchors1))
print(len(test_anchors))

for anchor in test_diff_anchors:
    print(f'{anchor}\n\n')

[]
322
438
438
438
<a href="/en-gb/research-and-development/partnerships/our-areas-of-interest/" itemprop="url">
																					Our areas of interest
																				</a>


<a href="/en-gb/research-and-development/research/how-we-research-new-vaccines/" itemprop="url">
																					How we research new vaccines
																				</a>


<a href="/en-gb/media/media-contact-details/">
<span>
																Media contact details
															</span>
</a>


<a href="/en-gb/responsibility/our-people/">
																			Visit this section
																		</a>


<a href="/en-gb/research-and-development/development/how-we-develop-new-medicines/" itemprop="url">
																					How we develop new medicines
																				</a>


<a href="/en-gb/about-us/comic-relief-partnership/tanzania/" itemprop="url">
																					Tanzania
																				</a>


<a href="/en-gb/investors/corporate-reporting/">
<span>
																Corporate reporting
										

In [15]:
for anchor in anchors2:
    print(f'{anchor}\n\n')

<a class="gsk-logo__link" href="/en-gb/">
<span class="gsk-logo__text">GSK logo linking to the homepage</span>
</a>


<a class="secondary-nav__bts" href="/en-gb/behind-the-science/" itemprop="url">
<span class="banner-hidden-trigger banner-hidden-trigger--mobile">
<div id="icon-dna--mobile"></div>
<strong>Behind the science</strong>
</span>
<span class="banner-hidden">
<span class="banner__content">
							Explore innovations in medicines, vaccines and consumer healthcare from inside and outside our labs 
							<svg class="icon-mini-chevron"><use xlink:href="/assets/img/icon-sprite.svg#icon-mini-chevron"></use>
</svg></span>
</span>
<span class="banner-hidden-trigger banner-hidden-trigger--desktop">
<div id="icon-dna--desktop">
<video autoplay="" height="32" loop="" muted="" width="20">
<source src="/assets/img/GSK_DNA_Icon_64x40.mp4" type="video/mp4"/>
</video>
</div>
<strong class="banner__label">
						Behind the science
					</strong>
</span>
<div class="banner-hidden-trigger-bg">

In [28]:
counter = 0
for anchor in test_anchors:
    print(counter)
    print(f'{anchor}\n\n')
    counter+=1

0
<a class="gsk-logo__link" href="/en-gb/">
<span class="gsk-logo__text">GSK logo linking to the homepage</span>
</a>


1
<a class="secondary-nav__bts" href="/en-gb/behind-the-science/" itemprop="url">
<span class="banner-hidden-trigger banner-hidden-trigger--mobile">
<div id="icon-dna--mobile"></div>
<strong>Behind the science</strong>
</span>
<span class="banner-hidden">
<span class="banner__content">
							Explore innovations in medicines, vaccines and consumer healthcare from inside and outside our labs 
							<svg class="icon-mini-chevron"><use xlink:href="/assets/img/icon-sprite.svg#icon-mini-chevron"></use>
</svg></span>
</span>
<span class="banner-hidden-trigger banner-hidden-trigger--desktop">
<div id="icon-dna--desktop">
<video autoplay="" height="32" loop="" muted="" width="20">
<source src="/assets/img/GSK_DNA_Icon_64x40.mp4" type="video/mp4"/>
</video>
</div>
<strong class="banner__label">
						Behind the science
					</strong>
</span>
<div class="banner-hidden-trigger-

In [ ]:
# test_load1 = load_cache(url_pr)
# test_load2 = load_cache(url_pr)
# test_soup1 = BeautifulSoup(test_load1, 'html.parser')
# test_soup2 = BeautifulSoup(test_load2, 'html.parser')
# test_anchors1 = get_anchors(test_soup1)
# test_href1, _ = parse_anchors(test_anchors1)
# test_anchors2 = get_anchors(test_soup2)
# test_diff_href = get_new_diff(href1, test_href1)
# print(test_diff_href)

a = ['1    2', '2 4', '3         5', 'sad\tasdf']
for i, val in enumerate(a):
    while '  ' in val or '\t' in val:
        val=val.replace('  ', ' ').replace('\t', ' ')
        a[i]=val

print(a)

In [ ]:
#AA = ['1    2', '2 4', '3         5', 'sad\tasdf']
AA = anchors2[0:2]
for i, a in enumerate(AA):
    a = str(a)
    count = 0
    while '  ' in a or '\t' in a or '\n' in a:
        # print('Found chars')
        a=a.replace('  ', ' ').replace('\t', ' ').replace('\n', ' ')
    AA[i] = a
print(AA)


In [73]:
import re
re.sub(' +', ' ', 'The     quick brown    fox')
'The quick brown fox'
print(test_diff_anchors[0] in anchors2)
print(test_diff_anchors[0] in test_anchors)
print('\n\n')
print(test_diff_anchors[0])
print('\n')
print(test_anchors[205])
print('\n\n')
# a = re.sub(' +', ' ', test_anchors[205].text.replace('\t', '    '))
# b = re.sub(' +', ' ', test_diff_anchors[0].text.replace('\t', '    '))

a = test_anchors[205].text.strip()
b = test_diff_anchors[0].text.strip()
print(len(a))
print(len(b))
print(a==b)

True
False



<a href="/en-gb/research-and-development/partnerships/our-areas-of-interest/" itemprop="url">
																					Our areas of interest
																				</a>


<a href="/en-gb/research-and-development/partnerships/our-areas-of-interest/" itemprop="url">
																					Our areas of interest
																				</a>



21
21
True


In [80]:
# count = 0
# while count < 25:
#     print(f'COUNT: {count}')
#     print(a[count])
#     print(b[count])
#     print('------------')
#     count += 1

In [89]:
a = '\t \t asdlkfjasd sad fasdfj dfsa   sakdlfja sdja \t adasdlkfjasldfj'
print(" ".join(a.split()))

asdlkfjasd sad fasdfj dfsa sakdlfja sdja adasdlkfjasldfj
